In [12]:
import numpy as np
import os
import nibabel as nib
from dipy.io.streamline import load_tractogram, save_tck
from dipy.tracking.streamline import set_number_of_points
from dipy.align.streamlinear import StreamlineLinearRegistration
from dipy.tracking.streamline import transform_streamlines
from dipy.tracking.streamline import (set_number_of_points, unlist_streamlines,
                                      Streamlines)
from dipy.viz.streamline import (viz_two_bundles, viz_vector_field,
                                 viz_displacement_mag)
from dipy.align.streamwarp import (bundlewarp, bundlewarp_vector_filed,
                                   bundlewarp_shape_analysis)
from dipy.data import fetch_bundle_warp_dataset
from dipy.io.stateful_tractogram import Space, StatefulTractogram
from nibabel.streamlines import Tractogram
from dipy.io.stateful_tractogram import StatefulTractogram, Space
from time import time
from dipy.viz import window, actor
import glob

In [13]:
# Directory containing .tck files
tck_directory1 = r'C:\Users\tudor\Desktop\thesis\prepost\preop'
tck_directory2 = r'C:\Users\tudor\Desktop\thesis\prepost\postop'
save_directory1 = r'C:\Users\tudor\Desktop\thesis\prepost\preop-aligned'
save_directory2 = r'C:\Users\tudor\Desktop\thesis\prepost\postop-aligned'
reference_anatomy_path1 = r'C:\Users\tudor\Desktop\thesis\prepost\preop\fa.nii.gz'
reference_anatomy_path2 = r'C:\Users\tudor\Desktop\thesis\prepost\postop\fa.nii.gz'
reference_anatomy1 = nib.load(reference_anatomy_path1)
reference_anatomy2 = nib.load(reference_anatomy_path2)


In [14]:
# List and sort all .tck files in the directory
tck_files1 = sorted(glob.glob(os.path.join(tck_directory1, '*.tck')))
tck_files2 = sorted(glob.glob(os.path.join(tck_directory2, '*.tck')))
tck_files_iterator = iter(tck_files2)

In [17]:
for file1 in tck_files1:
    file2 = next(tck_files_iterator)
    tractogram1 = load_tractogram(file1, reference_anatomy1)
    tractogram2 = load_tractogram(file2, reference_anatomy1)
    streamlines1 = set_number_of_points(tractogram1.streamlines, 100)
    streamlines2 = set_number_of_points(tractogram2.streamlines, 100)
    deformed_bundle, moving_aligned, distances, match_pairs, warp_map = bundlewarp(
                               streamlines2, streamlines1, alpha=0.5, beta=20, max_iter=15)
    
    affine = reference_anatomy2.affine
    tractogram = Tractogram(moving_aligned, affine_to_rasmm=affine)
    # Create a StatefulTractogram object
    sft = StatefulTractogram(tractogram.streamlines, reference_anatomy2, Space.RASMM)
    output_file = os.path.join(save_directory1, os.path.basename(file2).replace('.tck', '_aligned2.tck'))
    save_tck(sft, output_file, bbox_valid_check = False)
    
    tractogram2 = Tractogram(streamlines2, affine_to_rasmm=affine)
    # Create a StatefulTractogram object
    sft2 = StatefulTractogram(tractogram2.streamlines, reference_anatomy2, Space.RASMM)
    output_file2 = os.path.join(save_directory2, os.path.basename(file2).replace('.tck', '_aligned2.tck'))
    save_tck(sft2, output_file2, bbox_valid_check = False)

In [17]:
file1 = r'C:\Users\tudor\Desktop\thesis\prepost\preop\ATR_left.tck'
file2 = r'C:\Users\tudor\Desktop\thesis\prepost\postop\ATR_left.tck'
tractogram1 = load_tractogram(file1, reference_anatomy1)
tractogram2 = load_tractogram(file2, reference_anatomy1)
streamlines1 = set_number_of_points(tractogram1.streamlines, 100)
streamlines2 = set_number_of_points(tractogram2.streamlines, 100)
deformed_bundle, moving_aligned, distances, match_pairs, warp_map = bundlewarp(
                               streamlines2, streamlines1, alpha=0.5, beta=20, max_iter=15)
    
affine = reference_anatomy2.affine
tractogram = Tractogram(moving_aligned, affine_to_rasmm=affine)
# Create a StatefulTractogram object
sft = StatefulTractogram(tractogram.streamlines, reference_anatomy2, Space.RASMM)
output_file = os.path.join(save_directory1, os.path.basename(file2).replace('.tck', '_aligned.tck'))
save_tck(sft, output_file, bbox_valid_check = False)
    
tractogram2 = Tractogram(streamlines2, affine_to_rasmm=affine)
# Create a StatefulTractogram object
sft2 = StatefulTractogram(tractogram2.streamlines, reference_anatomy2, Space.RASMM)
output_file2 = os.path.join(save_directory2, os.path.basename(file2).replace('.tck', '_aligned.tck'))
save_tck(sft2, output_file2, bbox_valid_check = False)

# New alignement

In [ ]:
import nibabel as nib
import numpy as np
import os
from dipy.io.streamline import load_tractogram, save_tractogram
from dipy.io.stateful_tractogram import StatefulTractogram, Space
from dipy.align.streamlinear import whole_brain_slr
from dipy.tracking.streamline import transform_streamlines
import time

In [ ]:
tck_directory1 = r'C:\Users\tudor\Desktop\thesis\prepost\preop'
tck_directory2 = r'C:\Users\tudor\Desktop\thesis\prepost\postop'
reference_anatomy_path1 = r'C:\Users\tudor\Desktop\thesis\prepost\preop\fa.nii.gz'
reference_anatomy_path2 = r'C:\Users\tudor\Desktop\thesis\prepost\postop\fa.nii.gz'
reference_anatomy1 = nib.load(reference_anatomy_path1)
reference_anatomy2 = nib.load(reference_anatomy_path2)

In [ ]:
tck_files1 = sorted(glob.glob(os.path.join(tck_directory1, '*.tck')))
tck_files2 = sorted(glob.glob(os.path.join(tck_directory2, '*.tck')))
tck_files_iterator = iter(tck_files2)

In [ ]:
len(tck_files2)

In [14]:
def get_all_points(streamlines):
    return np.concatenate(streamlines, axis=0)

In [15]:
def compute_principal_axes(points_centered):
    cov_matrix = np.cov(points_centered.T)
    eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)
    # Sort eigenvectors by descending eigenvalues
    idx = eigenvalues.argsort()[::-1]
    eigenvectors = eigenvectors[:, idx]
    return eigenvectors

In [48]:
meanerror = []

In [49]:
for file1 in tck_files1:
    file2 = next(tck_files_iterator)
    tractogram1 = load_tractogram(file1, reference_anatomy1)
    tractogram2 = load_tractogram(file2, reference_anatomy1)
    streamlines_preop = tractogram1.streamlines
    streamlines_postop = tractogram2.streamlines
    points_preop = np.concatenate(streamlines_preop, axis=0)
    points_postop = np.concatenate(streamlines_postop, axis=0)
    transformation = np.eye(4)
    R, t, errors = icp(points_preop, points_postop)

    meanerror.append(errors[-1])
    print("ICP completed.")
    print(f"Final mean error: {errors[-1]}")
    print("Rotation matrix:")
    print(R)
    print("Translation vector:")
    print(t)
    
    # Apply the transformation to the postoperative streamlines
    transformed_streamlines_postop = []
    for streamline in streamlines_postop:
        transformed_streamline = np.dot(streamline, R.T) + t
        transformed_streamlines_postop.append(transformed_streamline)
    
    # Create a new StatefulTractogram with the transformed streamlines
    transformed_tractogram_postop = StatefulTractogram(
        transformed_streamlines_postop,
        reference_anatomy,
        Space.RASMM
    )
    
    # Save the translated tractogram
    output_path = os.path.join(r'C:\Users\tudor\Desktop\thesis\prepost\postop-aligned-v2', os.path.basename(file2).replace('.tck', '_aligned.tck'))
    save_tck(transformed_tractogram_postop, output_path, bbox_valid_check=False)


ICP completed.
Final mean error: 1.7340630973713045
Rotation matrix:
[[ 0.94251036 -0.33170572 -0.04056839]
 [ 0.33290185  0.9425655   0.02733627]
 [ 0.02917072 -0.03926992  0.99880285]]
Translation vector:
[-39.26393254 -25.67633201  -8.41948202]
ICP completed.
Final mean error: 1.2803423424121805
Rotation matrix:
[[ 0.36029129  0.13544279  0.92295514]
 [ 0.73067175  0.57411173 -0.36948073]
 [-0.57992242  0.80749703  0.10788321]]
Translation vector:
[20.18969896 25.76789053 22.36860182]
ICP completed.
Final mean error: 13.587754146416538
Rotation matrix:
[[ 0.86599408 -0.3589034   0.34819938]
 [ 0.36923815  0.92852648  0.03875185]
 [-0.33722048  0.09500956  0.93661907]]
Translation vector:
[-32.34544112  26.36615133 -16.47037325]
ICP completed.
Final mean error: 1.0068241187254816
Rotation matrix:
[[ 0.62318733  0.01318152 -0.78196201]
 [-0.1779151   0.97603119 -0.12533708]
 [ 0.76156686  0.21723116  0.61059479]]
Translation vector:
[59.831566    3.3402219   8.27405517]
ICP completed.

StopIteration: 

In [52]:
file1 = r'C:\Users\tudor\Desktop\thesis\prepost\preop\IFO_left.tck'
file2 = r'C:\Users\tudor\Desktop\thesis\prepost\postop\IFO_left.tck'
tractogram1 = load_tractogram(file1, reference_anatomy1)
tractogram2 = load_tractogram(file2, reference_anatomy1)
streamlines_preop = tractogram1.streamlines
streamlines_postop = tractogram2.streamlines

points_preop = np.concatenate(streamlines_preop, axis=0)
points_postop = np.concatenate(streamlines_postop, axis=0)

transformation = np.eye(4)
R, t, errors = icp(points_preop, points_postop)

print("ICP completed.")
print(f"Final mean error: {errors[-1]}")
print("Rotation matrix:")
print(R)
print("Translation vector:")
print(t)

# Apply the transformation to the postoperative streamlines
transformed_streamlines_postop = []
for streamline in streamlines_postop:
    transformed_streamline = np.dot(streamline, R.T) + t
    transformed_streamlines_postop.append(transformed_streamline)

# Create a new StatefulTractogram with the transformed streamlines
transformed_tractogram_postop = StatefulTractogram(
    transformed_streamlines_postop,
    reference_anatomy,
    Space.RASMM
)

# Save the translated tractogram
#output_path = os.path.join(r'C:\Users\tudor\Desktop\thesis\prepost\postop-aligned-v2', os.path.basename(file2).replace('.tck', '_aligned.tck'))
output_path = os.path.join(
    r'C:\Users\tudor\Desktop\thesis\prepost\postop-aligned-v2',
    'IFO_left_aligned.tck'
)
save_tck(transformed_tractogram_postop, output_path, bbox_valid_check=False)

ICP completed.
Final mean error: 0.48195789120574006
Rotation matrix:
[[ 0.99303664  0.11287232  0.03373667]
 [-0.10776465  0.98603969 -0.12693575]
 [-0.04759314  0.12241617  0.9913372 ]]
Translation vector:
[  1.13751293   5.20136129 -12.88586955]


In [52]:
meanerror

[1.7340630973713045,
 1.2803423424121805,
 13.587754146416538,
 1.0068241187254816,
 11.867083046422835,
 0.8600142312766894,
 4.4408388042288465,
 1.5214862483041534,
 9.228802327284903,
 2.8754119372283857,
 4.859786667125143,
 1.6796917223327252,
 3.459276235800608,
 1.5219493110000994,
 2.9987041064860525,
 2.3294370977478214,
 3.20751255172355,
 1.3607507265721737,
 2.9691684294943124,
 10.282322379834806,
 7.109846986344646,
 1.077993058055363,
 5.610315538591065,
 4.139281922376531,
 3.4008398464660607,
 1.4680335721338555,
 1.5733111311277599,
 2.545761608759786,
 3.7077580101478103,
 1.025569158932891,
 1.2205922385827161,
 1.0700475876815474,
 1.8979985731702298,
 2.193733091978633,
 4.183072430661558,
 2.3371860725433433,
 2.765490531114484,
 1.3682904278144161,
 1.9064868273196858,
 1.5234289038732445,
 1.287885106748854,
 3.671304504698523,
 6.003521230120513,
 6.9990743791959265,
 4.224813369910229,
 2.183114808892821,
 1.448596839726622,
 2.158149576590704,
 2.4319103099

In [54]:
sum(meanerror) / len(meanerror)

3.135854578112822

# fml this is OLDEN

In [18]:
import nibabel as nib
import numpy as np
import os
from scipy.spatial import cKDTree
from dipy.io.streamline import load_tractogram, save_tractogram
from dipy.io.stateful_tractogram import StatefulTractogram, Space

# Paths to your tractography and reference files
tck_preop = r'C:\Users\tudor\Desktop\thesis\prepost\preop\CG_left.tck'
tck_postop = r'C:\Users\tudor\Desktop\thesis\prepost\postop\CG_left.tck'
reference_anatomy_path = r'C:\Users\tudor\Desktop\thesis\prepost\preop\fa.nii.gz'

# Load the reference anatomy image
reference_anatomy = nib.load(reference_anatomy_path)

# Load the tractograms
tractogram_preop = load_tractogram(tck_preop, reference_anatomy, bbox_valid_check=False)
tractogram_postop = load_tractogram(tck_postop, reference_anatomy, bbox_valid_check=False)

streamlines_preop = tractogram_preop.streamlines
streamlines_postop = tractogram_postop.streamlines

# Concatenate all points from streamlines to create point clouds
points_preop = np.concatenate(streamlines_preop, axis=0)
points_postop = np.concatenate(streamlines_postop, axis=0)

In [15]:
def icp(A, B, max_iterations=50, tolerance=1e-5):
    """
    Perform ICP to find the best rigid transformation that aligns point cloud B to point cloud A.

    Parameters:
    - A: Nx3 numpy array of points (fixed point cloud)
    - B: Nx3 numpy array of points (moving point cloud)
    - max_iterations: Maximum number of iterations
    - tolerance: Convergence criteria

    Returns:
    - R: 3x3 rotation matrix
    - t: 3x1 translation vector
    - errors: List of mean squared errors at each iteration
    """
    prev_error = None
    errors = []
    R = np.eye(3)
    t = np.zeros((3, 1))

    for i in range(max_iterations):
        tree = cKDTree(A)
        distances, indices = tree.query(B)
        A_matched = A[indices]
        B_matched = B
        centroid_A = np.mean(A_matched, axis=0)
        centroid_B = np.mean(B_matched, axis=0)
        AA = A_matched - centroid_A
        BB = B_matched - centroid_B
        H = np.dot(BB.T, AA)
        U, S, Vt = np.linalg.svd(H)
        R_iter = np.dot(Vt.T, U.T)
        if np.linalg.det(R_iter) < 0:
            Vt[-1, :] *= -1
            R_iter = np.dot(Vt.T, U.T)
        t_iter = centroid_A.T - np.dot(R_iter, centroid_B.T)
        B = np.dot(B, R_iter.T) + t_iter.T
        R = np.dot(R_iter, R)
        t = np.dot(R_iter, t) + t_iter.reshape(3, 1)

        # Compute mean error
        mean_error = np.mean(distances)
        errors.append(mean_error)

        if prev_error is not None and abs(prev_error - mean_error) < tolerance:
            break
        prev_error = mean_error

    return R, t.flatten(), errors

In [17]:
transformation = np.eye(4)


# Perform ICP to find the optimal rotation and translation
R, t, errors = icp(points_preop, points_postop)

print("ICP completed.")
print(f"Final mean error: {errors[-1]}")
print("Rotation matrix:")
print(R)
print("Translation vector:")
print(t)

# Apply the transformation to the postoperative streamlines
transformed_streamlines_postop = []
for streamline in streamlines_postop:
    transformed_streamline = np.dot(streamline, R.T) + t
    transformed_streamlines_postop.append(transformed_streamline)

# Create a new StatefulTractogram with the transformed streamlines
transformed_tractogram_postop = StatefulTractogram(
    transformed_streamlines_postop,
    reference_anatomy,
    Space.RASMM
)

# Save the transformed tractogram
output_path = os.path.join(
    r'C:\Users\tudor\Desktop\thesis\prepost\postop-aligned-v2',
    'ATR_left_aligned.tck'
)
#save_tractogram(transformed_tractogram_postop, output_path, bbox_valid_check=False)

print(f"Transformed tractogram saved to {output_path}")

NameError: name 'cKDTree' is not defined

# rest of junk

In [51]:
# Paths to your tractography and reference files
tck_directory1 = r'C:\Users\tudor\Desktop\thesis\prepost\preop\IFO_left.tck'
tck_directory2 = r'C:\Users\tudor\Desktop\thesis\prepost\postop\IFO_left.tck'
reference_anatomy_path1 = r'C:\Users\tudor\Desktop\thesis\prepost\preop\fa.nii.gz'
reference_anatomy_path2 = r'C:\Users\tudor\Desktop\thesis\prepost\postop\fa.nii.gz'

# Load the reference anatomy images
reference_anatomy1 = nib.load(reference_anatomy_path1)
reference_anatomy2 = nib.load(reference_anatomy_path2)

# Load the tractograms
tractogram1 = load_tractogram(tck_directory1, reference_anatomy1, bbox_valid_check=False)
tractogram2 = load_tractogram(tck_directory2, reference_anatomy2, bbox_valid_check=False)

streamlines1 = tractogram1.streamlines
streamlines2 = tractogram2.streamlines

# Function to concatenate all points from streamlines into a single array
def get_all_points(streamlines):
    return np.concatenate(streamlines, axis=0)

# Get all points from both tractograms
points1 = get_all_points(streamlines1)
points2 = get_all_points(streamlines2)

# Compute centroids of both point clouds
centroid1 = np.mean(points1, axis=0)
centroid2 = np.mean(points2, axis=0)

# Center the point clouds
points1_centered = points1 - centroid1
points2_centered = points2 - centroid2

# Perform PCA to get principal axes
def compute_principal_axes(points_centered):
    cov_matrix = np.cov(points_centered.T)
    eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)
    # Sort eigenvectors by descending eigenvalues
    idx = eigenvalues.argsort()[::-1]
    eigenvectors = eigenvectors[:, idx]
    return eigenvectors

principal_axes1 = compute_principal_axes(points1_centered)
principal_axes2 = compute_principal_axes(points2_centered)

# Compute rotation matrix to align principal axes
R = np.dot(principal_axes2, principal_axes1.T)

# Ensure a proper rotation matrix (determinant = 1)
if np.linalg.det(R) < 0:
    R[:, -1] *= -1

# Apply rotation and translation to postoperative streamlines
transformed_streamlines2 = []
for s in streamlines2:
    s_centered = s - centroid2
    s_rotated = np.dot(s_centered, R)
    s_transformed = s_rotated + centroid1
    transformed_streamlines2.append(s_transformed)

# Create a new StatefulTractogram with the transformed streamlines
transformed_tractogram2 = StatefulTractogram(
    transformed_streamlines2,
    reference_anatomy1,  # Align to preoperative reference
    Space.RASMM
)

# Save the transformed tractogram
output_path = os.path.join(
    r'C:\Users\tudor\Desktop\thesis\prepost\postop-aligned-v2',
    'IFO_left_aligned.tck'
)
#save_tractogram(transformed_tractogram2, output_path, bbox_valid_check=False)

True

# renderstreamlines

In [53]:
tck_directory1 = r'C:\Users\tudor\Desktop\thesis\prepost\preop\IFO_left.tck'
tck_directory2 = r'C:\Users\tudor\Desktop\thesis\prepost\postop-aligned-v2\IFO_left_aligned.tck'
reference_anatomy_path1 = r'C:\Users\tudor\Desktop\thesis\prepost\preop\fa.nii.gz'
reference_anatomy_path2 = r'C:\Users\tudor\Desktop\thesis\prepost\postop\fa.nii.gz'
reference_anatomy1 = nib.load(reference_anatomy_path1)
reference_anatomy2 = nib.load(reference_anatomy_path2)


uf_subj1 = load_tractogram(tck_directory1,reference_anatomy1,
                    bbox_valid_check=False).streamlines
uf_subj2 = load_tractogram(tck_directory2, reference_anatomy2,
                    bbox_valid_check=False).streamlines

In [54]:
len(uf_subj2)


3000

In [55]:
import nibabel as nib
from dipy.io.streamline import load_tractogram
from fury import window, actor

In [56]:
def render_streamlines(streamlines, title, color):
    scene = window.Scene()
    stream_actor = actor.line(streamlines, colors=color)
    scene.add(stream_actor)
    
    window.show(scene, size=(800, 800), title=title)

# Render each streamline in a separate scene
render_streamlines(uf_subj1, title="Preoperative AF Left", color=(1, 0, 0))
render_streamlines(uf_subj2, title="Postoperative AF Left", color=(0, 0, 1))

# Render superimposed streamlines
def render_superimposed(streamlines1, streamlines2):
    scene = window.Scene()

    stream_actor1 = actor.line(streamlines1, colors=(1, 0, 0))  # Red color for preoperative
    stream_actor2 = actor.line(streamlines2, colors=(0, 0, 1))  # Blue color for postoperative

    scene.add(stream_actor1)
    scene.add(stream_actor2)

    window.show(scene, size=(800, 800), title="Superimposed Streamlines")

# Superimpose the streamlines
render_superimposed(uf_subj1, uf_subj2)

In [4]:
static = Streamlines(set_number_of_points(uf_subj1, 50))
moving = Streamlines(set_number_of_points(uf_subj2, 50))

In [5]:
deformed_bundle, moving_aligned, distances, match_pairs, warp_map = bundlewarp(
                               static, moving, alpha=0.5, beta=20, max_iter=15)

In [6]:
tck_directory3 = r'C:\Users\tudor\Desktop\thesis\prepost\postop\ATR_right_deformed4.tck'
uf_subj3 = load_tractogram(tck_directory3, reference=reference_anatomy_path2,
                    bbox_valid_check=False).streamlines
helpt = Streamlines(set_number_of_points(uf_subj3, 50))

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\tudor\\Desktop\\thesis\\prepost\\postop\\ATR_right_deformed4.tck'

In [31]:
viz_two_bundles(static, helpt,
                fname="huh.png")

In [ ]:
scene = window.Scene()

# Add the static streamlines (streamlines1) to the scene
scene.add(actor.line(uf_subj3, colors=(1, 0, 0), opacity=0.5))

# Add the aligned streamlines (aligned_streamlines2) to the scene
scene.add(actor.line(uf_subj2, colors=(0, 0, 1), opacity=0.5))

# Show the scene
window.show(scene)

In [18]:
affine = reference_anatomy2.affine

tractogram = Tractogram(moving_aligned, affine_to_rasmm=affine)

# Create a StatefulTractogram object
sft = StatefulTractogram(tractogram.streamlines, reference_anatomy2, Space.RASMM)

save_tck(sft, r'C:\Users\tudor\Desktop\thesis\prepost\postop\CST_left_aligned.tck', bbox_valid_check = False)

In [19]:
tractogram2 = Tractogram(static, affine_to_rasmm=affine)

# Create a StatefulTractogram object
sft2 = StatefulTractogram(tractogram2.streamlines, reference_anatomy2, Space.RASMM)

save_tck(sft2, r'C:\Users\tudor\Desktop\thesis\prepost\postop\CST_left_points.tck', bbox_valid_check = False)

In [20]:
tractogram3 = Tractogram(deformed_bundle, affine_to_rasmm=affine)

# Create a StatefulTractogram object
sft3 = StatefulTractogram(tractogram3.streamlines, reference_anatomy2, Space.RASMM)

save_tck(sft3, r'C:\Users\tudor\Desktop\thesis\prepost\postop\CST_left_deformed.tck', bbox_valid_check = False)